# Personalized Marketing Campaign Content Creation
## Use case

Assuming you are a marketing campaign manager, you're going to promote the flight ticket for Airline. At first, AI system will help you find out the target users segment; and then generate marketing promotion template for thoes target users. 
There has two AI engines
1. Recommendation engine - Amazon Personalize service
2. Content Generative engine - Amazon Bedrock service


The pipeline - 

Marketing request-->Personalize-->retrive medata-->combine with PromptTemplate--> Langchain-->Amazon Bedrock & LLM--> Generate content -->save in JSON

## Introduction

In this notebook we show how to generate personalized marketing campaign promotion for email, by contextual metadata and question template. 

LangChain is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example creates a custom LangChain prompt template for adding context to the text generation request. 


LangChain is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Models and enable us to perform tasks which meet desired goals and unlock various use-cases.



#### Pre-requisites
Before we get started with the implementation we have to make sure that the required boto3 and botocore packages are installed. These will be used to leverage the Amazon Bedrock API client.

Additionally we would need langchain one of  the latest versions 0.0.190 which has Amazon Bedrock class implemented under llms module. Also we are installing the transformers framework from HuggingFace, which we will use to quickly count the number of tokens in the input prompt.

In [ ]:
%pip install --upgrade pip

In [ ]:
!python3 -m pip install boto3-1.26.162-py3-none-any.whl
!python3 -m pip install botocore-1.29.162-py3-none-any.whl

In [ ]:
%pip install langchain==0.0.190 --quiet
%pip install transformers==4.24.0 --quiet

In [ ]:
%pip install ipywidgets

# install timer counter 
%pip install tqdm


In [ ]:
import boto3
import json
import ipywidgets as widgets
from IPython.display import display
from langchain import PromptTemplate

import os
import sys

import time
from tqdm import tqdm

boto3_bedrock  = None


## Prepare Content generative engine, invoke the Bedrock LLM Model

for more details for the parameters please refer to the bedrock api page

In [ ]:
bedrock = boto3.client(
 service_name='bedrock',
 region_name='us-east-1',
 endpoint_url='https://bedrock.us-east-1.amazonaws.com'
)

In [ ]:
bedrock.list_foundation_models()

In [ ]:
from langchain.llms.bedrock import Bedrock

# textgen_llm = Bedrock(model_id="amazon.titan-tg1-large", client=boto3_bedrock)

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.7,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


## Prepare contextual item medata, and question template 

In [ ]:
def read_json_data(file_path):
    with open(file_path, "r") as json_file:
        data = json.load(json_file)
    return data

def on_confirm_button_click(b):
    selected_metadata_file = metadata_dropdown.value
    selected_template_file = template_dropdown.value

    metadata = read_json_data(selected_metadata_file)
    print("Metadata JSON data:")
    print(json.dumps(metadata, indent=4))

    template = read_json_data(selected_template_file).get("Question", "")
    print("Ticketing Template:")
    print(template)

metadata_dropdown = widgets.Dropdown(
    options=["test-metadata.json", "other-metadata.json"],  # Add more options if needed
    description="Metadata File:"
)

template_dropdown = widgets.Dropdown(
    options=["ticketing-template.json", "other-template.json"],  # Add more options if needed
    description="Template File:"
)

confirm_button = widgets.Button(description="Confirm button")
confirm_button.on_click(on_confirm_button_click)

display(metadata_dropdown, template_dropdown, confirm_button)



## Prepare the prompting template by metadata and question template

In [ ]:
def update_output():
    with open("test-metadata.json", "r") as json_file:
        metadata = json.load(json_file)

    with open("ticketing-template.json", "r") as json_file:
        ticketing_template = json.load(json_file)

    input_variables = list(metadata.keys())
    template = ticketing_template.get("Question", "")

    multi_var_prompt = PromptTemplate(
        input_variables=input_variables,
        template=template
    )

    prompt = multi_var_prompt.format(**metadata)

    output.clear_output()
    with output:
        print(prompt)
    
    
    return prompt    


In [ ]:
# Create the output widget
output = widgets.Output()

# Create the button widget
button = widgets.Button(description="Generate Question")

# Define the event handler for the button
button.on_click(update_output)

# Display the button and output widget
display(button, output)

prompt = update_output()

### Calculate the number of input tokens

In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

## Generate the content by the request of prompting question

invoke using the prompt tempalate and expect to see a curated response back

In [ ]:

response = textgen_llm(prompt)

# Measure the time taken for text generation
start_time = time.time()
email = response[response.index('\n')+1:]
end_time = time.time()
responding_time = end_time - start_time

# Print the responding time
print("Responding time:", responding_time, "seconds")

print("\nEmail generation complete.")


In [ ]:
print(email)

## Optional: Prompt Chaining

Prompt chaining can allow you to accomplish a complex task by passing Claude multiple smaller and simpler prompts instead of a very long and detailed one. It can sometimes work better than putting all of a task's subtasks in a single prompt.

Turning a long and complex prompt into a prompt chain can have a few advantages:

You can write less complicated instructions.
You can isolate parts of a problem that Claude is having trouble with to focus your troubleshooting efforts.
You can check Claude's output in stages, instead of just at the end.

https://docs.anthropic.com/claude/docs/prompt-chaining

In [ ]:
import json

# Load the JSON file
with open('top50inHK.json', 'r') as file:
    data = json.load(file)

# Define a prompt and additional statement

additional_statement = "These are the top 50 most fun and delicious places to recommend in Hong Kong, please enrich the itinerary, and generate again"

# Initialize empty lists for the title and body of each paragraph
title_paragraphs = []
body_paragraphs = []

# Loop through each category and its items
for category, items in data.items():
    # Combine the category and items into a single string for the title
    title = f"{category}: {', '.join(items)}"
    # Create a body paragraph with a brief description or additional information
    #body = "Explore these fantastic places to make the most of your trip to Hong Kong."

    # Append the title and body paragraphs to their respective lists
    title_paragraphs.append(title)
    #body_paragraphs.append(body)

# Combine paragraphs with prompt and additional statement
combined_paragraphs = "\n\n".join(title_paragraphs + [additional_statement] + body_paragraphs)
promptChaining = f"{prompt}\n\n{combined_paragraphs}"

# Print the final prompt
print(promptChaining)


In [ ]:

response = textgen_llm(promptChaining)

# Measure the time taken for text generation
start_time = time.time()
email2 = response[response.index('\n')+1:]
end_time = time.time()
responding_time = end_time - start_time

# Print the responding time
print("Responding time:", responding_time, "seconds")

print("\nEmail generation complete.")

In [ ]:
 print(email2)

## Save the content in JSON file

In [ ]:
import json

# Assuming you already have the 'email' variable with the email content
# If not, you can replace this with the appropriate content.

email_content = email.strip()

# Create a dictionary with the email content
email_dict = {"email": email_content}

# Define the filename for the JSON file
json_filename = "email_content_english.json"

# Save the dictionary as a JSON file
with open(json_filename, "w") as json_file:
    json.dump(email_dict, json_file, indent=4)

print(f"The email content has been saved as {json_filename}.")

## Text to Image (optional)

In [ ]:
%pip install --quiet "pillow>=9.5,<10"

In [ ]:
import base64
import io
import json
import os
import sys

import boto3
from PIL import Image
bedrock_client = boto3.client('bedrock' , 'us-east-1', endpoint_url='https://bedrock.us-east-1.amazonaws.com')
bedrock_client.list_foundation_models()

## Prepare prompting


In [ ]:
print (prompt)

In [ ]:

negative_prompts = [
    "poorly rendered",
    "poor background details",
    "Not realistic enough",
    "poorly drawn people",
    "poorly drawn human eyes",
    "poorly drawn human nose",
    "poorly drawn human hair",
    "poorly drawn human finger",
    "no landmark",
    "missing human fingers",
    "poorly drawn aircraft",
    "poorly quality of font",
    "pixels too lower",
    "city looks not prosperous enough"
    "disfigured people features",
]
style_preset = "photographic"  # (e.g. photographic, digital-art, cinematic, ...)
#prompt = "photo taken from above of an italian landscape. cloud is clear with few clouds. Green hills and few villages, a lake"

### The Stability.ai Diffusion models support the following controls. 

cfg_scale: Prompt strength– Determines how much the final image portrays the prompt random generations. The range is 0—30, and the default value is 10.
the "cfg_scale" essentially governs how much the image looks closer to the prompt or input image. The higher the CFG scale, the more the image will match your prompt. Conversely, a lower CFG scale value produces a better-quality image that may differ from the original prompt or image

In Stable Diffusion, CFG stands for Classifier Free Guidance scale. CFG is the setting that controls how closely Stable Diffusion should follow your text prompt. It is applied in text-to-image (txt2img) and image-to-image (img2img) generations.

The higher the CFG value, the more strictly it will follow your prompt, in theory. The default value is 10, which gives a good balance between creative freedom and following your direction. A value of 1 will give Stable Diffusion almost complete freedom, whereas values above 15 are quite restrictive.

step: Generation step determines how many times the image is sampled. More steps can result in a more accurate result. The range is 0—150, and the default value is 5.

Seed: The seed determines the initial noise setting. If you use the same seed and the same settings as a previous run, inference creates a similar image. The seed value is a random number.


style_preset: the parameter includes enhance, anime, photographic, digital-art, comic-book, fantasy-art, line-art, analog-film, neon-punk, isometric, low-poly, origami, modeling-compound, cinematic, 3d-model, pixel-art, and tile-texture. This list of style presets is subject to change; refer to the latest release and documentation for updates.

https://platform.stability.ai/docs/api-reference#tag/v1generation/operation/textToImage

In [ ]:
import json
import random

# Generate a random seed value
random_seed = random.randint(1, 9999999)  # Adjust the range as needed

request = json.dumps({
    "text_prompts": (
        [{"text": prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": 10,
    "seed": random_seed,  # Assign the random seed value here
    "steps": 150,
    "style_preset": style_preset,
})
modelId = "stability.stable-diffusion-xl"

response = bedrock_client.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")


### By decoding our Base64 string to binary, and loading it with an image processing library like Pillow that can read PNG files, we can display and manipulate the image here in the notebook:

In [ ]:
os.makedirs("data", exist_ok=True)
image_1 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
image_1.save("data/image_1.png")
image_1

## Image to Image

Generating images from text is powerful, but in some cases could need many rounds of prompt refinement to get an image "just right".

Rather than starting from scratch with text each time, image-to-image generation lets us modify an existing image to make the specific changes we'd like.

We'll have to pass our initial image in to the API in base64 encoding, so first let's prepare that. You can use either the initial image from the previous section, or a different one if you'd prefer:

In [ ]:
def image_to_base64(img) -> str:
    """Convert a PIL Image or local image file path to a base64 string for Amazon Bedrock"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        print("Converting PIL Image to base64 string")
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")




### Use the sample image for new image

In [ ]:
# Define image_to_image1 as a PIL Image object with your actual image file path)
image_to_image1 = Image.open('image_to_image1.png')

init_image_b64 = image_to_base64(image_to_image1)
print(init_image_b64[:80] + "...")

In [ ]:
image_to_image1

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": 10,
    "init_image": init_image_b64,
    "seed": 3661,
    "start_schedule": 0.6,
    "steps": 150,
    "style_preset": style_preset,
})
modelId = "stability.stable-diffusion-xl"

response = bedrock_client.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
image_2_b64_str = response_body["artifacts"][0].get("base64")
print(f"{image_2_b64_str[0:80]}...")

In [ ]:
image_2 = Image.open(io.BytesIO(base64.decodebytes(bytes(image_2_b64_str, "utf-8"))))
image_2.save("data/image_2.png")
image_2